## 컨볼루션 신경망 레이어 이야기

In [ ]:
Conv2D(32, (5, 5), padding='valid', input_shape=(28, 28, 1), activation='relu')

## 주요 인자
- ### 첫 번째 인자: 컨볼루션 필터의 수
    - #### 필터 수에 따라 출력 이미지 수가 결정된다
- ### 두 번째 인자: 컨볼루션 커널의 (행, 열)
- ### padding: 경계 처리 방법을 정의
    - #### 'valid': 유효한 영역만 출력됩니다. 따라서 출력 이미지 사이즈는 입력 이미지 사이즈보다 작습니다.
    - #### 'same': 출력 이미지 사이즈가 입력 이미지 사이즈와 동일합니다.
- ### input_shape: 샘플 수를 제외한 입력 형태를 정의합니다. 모델에서 첫 레이어일 때만 정의하면 됩니다.
    - #### (행, 열, 채널 수)로 정의합니다. 흑백 영상인 경우에는 채널이 1이고, 컬러(RGB) 영상인 경우에는 채널을 3으로 설정합니다.

In [ ]:
MaxpPooling2D(pool_size=(2,2))
    # 가장 큰 값만 뽑아내고 차원을 축소시킨다 (추상화 과정)

## Flatten()
- ### 2차원 벡터를 1차원으로 만든다

In [ ]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
np.random.seed(3)

# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'warehouse/handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'warehouse/handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        # fit 함수는 x를 넘기지만 fit_generator는 제네레이터를 넘긴다
        train_generator,
        steps_per_epoch=15,
        epochs=50,
        validation_data=test_generator,
        validation_steps=5)

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)()